# Model Serialization

## Tổng quan

Serialization (tuần tự hóa) là quá trình chuyển đổi một object thành một định dạng có thể dễ dàng được lưu trữ, chia sẻ hoặc truyền tải, và sau đó được tái tạo. Trong framework LangChain, các class triển khai các phương thức tiêu chuẩn cho serialization, mang lại một số lợi ích:

- **Separation of Secrets**: Thông tin nhạy cảm, chẳng hạn như API keys, được tách biệt khỏi các parameters khác và có thể được tải lại một cách an toàn vào object trong quá trình deserialization (giải tuần tự hóa).

- **Version Compatibility**: Deserialization vẫn tương thích giữa các phiên bản package khác nhau, đảm bảo rằng các object được serialized bằng một phiên bản của LangChain có thể được deserialized đúng cách bằng một phiên bản khác.

Tất cả các object LangChain kế thừa từ `Serializable` đều có thể được JSON-serialized, bao gồm messages, document objects (ví dụ: những object được trả về từ retrievers), và hầu hết các Runnables như chat models, retrievers và chains được triển khai bằng LangChain Expression Language.

### Lưu và Tải các Object LangChain

Để quản lý hiệu quả các object LangChain, bạn có thể serialize và deserialize chúng bằng các functions sau:

- **`dumpd`**: Trả về một dictionary representation của một object, phù hợp cho JSON serialization.

- **`dumps`**: Trả về một JSON string representation của một object.

- **`load`**: Tái tạo một object từ dictionary representation của nó.

- **`loads`**: Tái tạo một object từ JSON string representation của nó.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate.from_template("Liệt kê 5 món ăn ngon ở {place}")
chain = prompt | llm | StrOutputParser()

answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.
2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc biệt.
3. Bánh mỳ pate: Một món ăn sáng ngon và phổ biến ở Hà Nội, bánh mỳ pate thường được ăn kèm với pate, thịt nguội, rau sống và sốt.
4. Nem chua rán: Một món ăn vặt ngon và hấp dẫn, nem chua rán là sự kết hợp giữa nem chua và chả rán, tạo nên hương vị độc đáo.
5. Bún ốc: Một món ăn truyền thống của Hà Nội, bún ốc gồm bún, ốc và nước dùng thơm ngon, tạo nên một món ăn ngon và lạ miệng.


## `Dumps` and `Loads`

- `dumps` : LangChain object into a JSON-formatted string
- `loads` : JSON-formatted string into a LangChain object

In [2]:
from langchain_core.load.dump import dumps
serialized_llm = dumps(llm, pretty=True)
print(type(serialized_llm))
print(serialized_llm)

<class 'str'>
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "chat_models",
    "openai",
    "ChatOpenAI"
  ],
  "kwargs": {
    "model_name": "gpt-3.5-turbo",
    "temperature": 0.0,
    "openai_api_key": {
      "lc": 1,
      "type": "secret",
      "id": [
        "OPENAI_API_KEY"
      ]
    }
  },
  "name": "ChatOpenAI"
}


In [3]:
serialized_prompt = dumps(prompt, pretty=True)
print(type(serialized_prompt))
print(serialized_prompt[:100] + " ...")

<class 'str'>
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    "prompts",
    "prompt",
    "P ...


In [4]:
serialized_chain = dumps(chain)
print(type(serialized_chain))
print(serialized_chain[:100] + " ...")

<class 'str'>
{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "runnable", "RunnableSequence"], "kwa ...


In [5]:
from langchain_core.load.load import loads

# Deserialize JSON like string to LangChain object

deserialized_llm = loads(serialized_llm)
print(type(deserialized_llm))

deserialized_prompt = loads(serialized_prompt)
print(type(deserialized_prompt))

deserialized_chain = loads(serialized_chain)
print(type(deserialized_chain))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>
<class 'langchain_core.runnables.base.RunnableSequence'>


/tmp/ipykernel_535673/1905072887.py:5: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  deserialized_llm = loads(serialized_llm)


In [8]:
answer = deserialized_chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và thơm.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate được làm từ bánh mỳ nướng giòn và pate thơm ngon.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua cay, giòn ngon và thích hợp để thưởng thức cùng bia.

5. Chả cá Lã Vọng: Một món ăn đặc sản của Hà Nội, chả cá Lã Vọng gồm cá chẽm nướng trên bếp than, ăn kèm với bún, rau sống và nước mắm chua ngọt.


In [9]:
response = (deserialized_prompt | deserialized_llm).invoke("Hà Nội")
print(response.content)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và thơm.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate được làm từ bánh mỳ nướng giòn, phết pate và thêm rau sống, dưa leo, chả lụa, tạo nên hương vị độc đáo.

4. Nem chua rán: Một món ăn vặt ngon và hấp dẫn của Hà Nội, nem chua rán là sự kết hợp giữa nem chua và bột chiên giòn, tạo nên hương vị đặc trưng và ngon miệng.

5. Chả cá Lã Vọng: Một món ăn đặc sản của Hà Nội, chả cá Lã Vọng gồm cá lăng nướng trên bếp than, ăn kèm với bún, rau sống và nước mắm chua ngọt, tạo nên hương vị độc đáo và ngon tuyệt.


## `Dumpd` and `Load`

- `dumpd` : LangChain object into a dictionary
- `load` : dictionary into a LangChain object


In [10]:
from langchain_core.load.dump import dumpd

# Serialize LangChain object to dictionary

serialized_llm = dumpd(llm)
print(type(serialized_llm))

serialized_prompt = dumpd(prompt)
print(type(serialized_prompt))

serialized_chain = dumpd(chain)
print(type(serialized_chain))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [11]:
from langchain_core.load.load import load

# Deserialize dictionary to LangChain object

deserialized_llm = load(serialized_llm)
print(type(deserialized_llm))

deserialized_prompt = load(serialized_prompt)
print(type(deserialized_prompt))

deserialized_chain = load(serialized_chain)
print(type(deserialized_chain))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>
<class 'langchain_core.runnables.base.RunnableSequence'>


/tmp/ipykernel_535673/2225592789.py:5: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  deserialized_llm = load(serialized_llm)


In [12]:
deserialized_chain.invoke("Hà Nội")

'1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.\n2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.\n3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.\n4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, ngọt, cay và giòn rất hấp dẫn.\n5. Bún ốc: Một món ăn truyền thống của Hà Nội, bún ốc có hương vị đặc trưng của ốc, nước dùng thơm ngon và bún mềm, tạo nên một món ăn ngon và lạ miệng.'

## Serialization with `pickle`

Module `pickle` trong Python được sử dụng để tuần tự hóa (serializing) và giải tuần tự hóa (deserializing) cấu trúc đối tượng Python, còn được gọi là _pickling_ và _unpickling_. Tuần tự hóa liên quan đến việc chuyển đổi một hệ thống phân cấp đối tượng Python thành một luồng byte, trong khi giải tuần tự hóa tái tạo hệ thống phân cấp đối tượng từ luồng byte.

[`pickle` - Python object serialization để biết thêm chi tiết]([https://docs.python.org/3/library/pickle.html](https://docs.python.org/3/library/pickle.html))

**Các Hàm Chính**

- **`pickle.dump(obj, file)`**: Tuần tự hóa `obj` và ghi nó vào đối tượng tệp đang mở `file`.

- **`pickle.load(file)`**: Đọc một luồng byte từ đối tượng tệp đang mở `file` và giải tuần tự hóa nó trở lại thành một đối tượng Python.


In [13]:
from langchain_core.load.dump import dumpd

# Serialize LangChain object to dictionary

serialized_llm = dumpd(llm)
print(type(serialized_llm))

serialized_prompt = dumpd(prompt)
print(type(serialized_prompt))

serialized_chain = dumpd(chain)
print(type(serialized_chain))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [14]:
import pickle
import os

# Serialize dictionary to pickle file

os.makedirs("data", exist_ok=True)

with open("data/serialized_llm.pkl", "wb") as f:
    pickle.dump(serialized_llm, f)

with open("data/serialized_prompt.pkl", "wb") as f:
    pickle.dump(serialized_prompt, f)

with open("data/serialized_chain.pkl", "wb") as f:
    pickle.dump(serialized_chain, f)

In [15]:
# Deserialize pickle file to dictionary

with open("data/serialized_llm.pkl", "rb") as f:
    loaded_llm = pickle.load(f)
    print(type(loaded_llm))

with open("data/serialized_prompt.pkl", "rb") as f:
    loaded_prompt = pickle.load(f)
    print(type(loaded_prompt))

with open("data/serialized_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)
    print(type(loaded_chain))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [16]:
from langchain_core.load.load import load

# Deserialize dictionary to LangChain object

deserialized_llm = load(loaded_llm)
print(type(deserialized_llm))

deserialized_prompt = load(loaded_prompt)
print(type(deserialized_prompt))

deserialized_chain = load(loaded_chain)
print(type(deserialized_chain))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
<class 'langchain_core.prompts.prompt.PromptTemplate'>
<class 'langchain_core.runnables.base.RunnableSequence'>


In [17]:
deserialized_chain.invoke("Hà Nội")

'1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.\n2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.\n3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.\n4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, ngọt, cay và giòn rất hấp dẫn.\n5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, được nhiều người yêu thích.'

## Is Every Runnable Serializable?

Các phương thức `dumps` và `dumpd` của LangChain cố gắng tuần tự hóa các đối tượng nhiều nhất có thể, nhưng dữ liệu kết quả có thể không đầy đủ.

1. Ngay cả khi phương thức `is_lc_serializable` không tồn tại hoặc trả về `False`, kết quả vẫn được tạo ra.
2. Ngay cả khi phương thức `is_lc_serializable` trả về `True` và quá trình tuần tự hóa thành công, quá trình giải tuần tự hóa có thể thất bại.

Sau khi tuần tự hóa, điều cần thiết là kiểm tra xem dữ liệu JSON có chứa `"type": "not_implemented"` hay không. Chỉ khi đó các hàm `load` hoặc `loads` mới có thể được sử dụng một cách an toàn.


In [20]:
from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.load.dump import dumps
from langchain_core.load.load import loads

chain = prompt | llm 

def custom_function(llm_response):
    return llm_response.content


# Define chains that make same results
chain_with_custom_function = chain | custom_function
print(type(chain_with_custom_function))
chain_with_str_output_parser = chain | StrOutputParser()
print(type(chain_with_str_output_parser))

response = chain_with_custom_function.invoke("Hà Nội")
print(response)

response = chain_with_str_output_parser.invoke("Hà Nội")
print(response)

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.runnables.base.RunnableSequence'>
1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và thơm.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, cay, ngọt và giòn, rất thích hợp để thưởng thức cùng bia.

5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, thường được ăn vào buổi tối.
1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon tuyệt vời.
2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua 

In [21]:
# Both of them are serializable
print(chain_with_custom_function.is_lc_serializable())
print(chain_with_str_output_parser.is_lc_serializable())

True
True


In [22]:
try:
    print(
        "...\n"
        # You can see that the serialized string contains "type": "not_implemented"
        + ((serialized_str := dumps(chain_with_custom_function, pretty=True)))[-270:]
    )
    # First one fail to deserialize
    loads(serialized_str)
except Exception as e:
    print("Error : \n", e)

print(type(deserialized_chain := loads(dumps(chain_with_str_output_parser))))
print(deserialized_chain.invoke("Hà Nội"))

...

    ],
    "last": {
      "lc": 1,
      "type": "not_implemented",
      "id": [
        "langchain_core",
        "runnables",
        "base",
        "RunnableLambda"
      ],
      "repr": "RunnableLambda(custom_function)"
    }
  },
  "name": "RunnableSequence"
}
Error : 
 Trying to load an object that doesn't implement serialization: {'lc': 1, 'type': 'not_implemented', 'id': ['langchain_core', 'runnables', 'base', 'RunnableLambda'], 'repr': 'RunnableLambda(custom_function)'}
<class 'langchain_core.runnables.base.RunnableSequence'>
1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.
2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.
3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.
4. Nem chua rán: Một món ăn vặt phổ biến và ngon miện

In [23]:
# RunnableLambda and custom_function has no is_lc_serializable method
# But they are serializable

try:
    print(RunnableLambda(custom_function).is_lc_serializable())
except Exception as e:
    print("Error : \n", e)

print(dumps(RunnableLambda(custom_function), pretty=True))

try:
    print(custom_function.is_lc_serializable())
except Exception as e:
    print("Error : \n", e)

print(dumps(custom_function, pretty=True))

Error : 
 'RunnableLambda' object has no attribute 'is_lc_serializable'
{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "runnables",
    "base",
    "RunnableLambda"
  ],
  "repr": "RunnableLambda(custom_function)"
}
Error : 
 'function' object has no attribute 'is_lc_serializable'
{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "__main__",
    "custom_function"
  ],
  "repr": "<function custom_function at 0x7b3ea0070a60>"
}


In [24]:
from langchain_core.load.serializable import Serializable

# Serializable has is_lc_serializable method
# But it returns False
print(Serializable.is_lc_serializable())

# But also it is serializable
print(dumps(Serializable, pretty=True))
print(dumpd(Serializable))

False
{
  "lc": 1,
  "type": "not_implemented",
  "id": [
    "langchain_core",
    "load",
    "serializable",
    "Serializable"
  ],
  "repr": "<class 'langchain_core.load.serializable.Serializable'>"
}
{'lc': 1, 'type': 'not_implemented', 'id': ['langchain_core', 'load', 'serializable', 'Serializable'], 'repr': "<class 'langchain_core.load.serializable.Serializable'>"}
